In [ ]:
import openai
import json
import functions.llm as llm
from openai.lib._parsing._completions import type_to_response_format_param
import pandas as pd
client = openai.Client()

In [ ]:
# G = ""
# with open('graph-relation.json', 'r') as f:
#     G = json.loads(f.read())
    
# ids = [*set([ node['id'] for node in G['nodes']])]

In [ ]:
model = "text-embedding-3-large"

def gen_body(text):
    return { "model": model, "input": text }

def wrap(id, body):
    return {
        "custom_id": id,
        "method": "POST",
        "url": "/v1/embeddings",
        "body": body,
    }


In [ ]:
# def batch():
#     reqs =  [ wrap(id, gen_body(id)) for id in ids ]
#     # return [ x for xs in reqs for x in xs ]
#     return reqs
# 
# with open(f"ids-{model}.jsonl", 'a') as f:
#     f.write(f"{"\n".join([json.dumps(i) for i in batch()])}\n")

In [ ]:
req_path = 'data/embeddings/req.jsonl'
llm.gen_batch_jsonl(req_path, [ wrap(id, gen_body(id)) for id in ids ])

# batch_input_file = client.files.create(
#     file=open(f"ids.jsonl", "rb"),
#     purpose="batch"
# )
# print(f"id: {batch_input_file.id}")

In [ ]:
batch = llm.gen_batch(client, req_path, metadata="Atlas of Human-AI Interaction [Embedding]")

In [ ]:
print(f"File ID: {batch['batch_file']}")
print(f"Batch ID: {batch['batch_object']}")

fetch.save(f"File ID: {batch['batch_file']} \nBatch ID: {batch['batch_object']}", "data/triplets/req_file_id.txt")

In [ ]:
client.batches.retrieve(batch['batch_object'])

In [ ]:
with open("embeddings.jsonl", "w") as f:
    f.write(client.files.content(r.output_file_id).text)

In [ ]:
df = pd.DataFrame(columns=["id", "embedding"])

with open("embeddings.jsonl", "r") as f:
    for i in f:
        l = json.loads(i)
        df.loc[len(df)] = [l['custom_id'], l['response']['body']['data'][0]['embedding']]

In [ ]:
df.to_csv("embeddings.csv")